Assume that the time that the firefighters arrive = start time + deployment time

Assume that it takes 4 hours to deal with low severity, 12 hours to deal with medium severity and 36 hours for high severity

Assume that the efficiency point (ep) is equal to the price (in thousand) of each unit, so Tanker Planes(15) > Helicopters (8) > Smoke Jumpers (5) > Groud Crews (3) > Fire Engines (2), and multiple units is avaiable for the same mission, and to be able to solve the low: 4ep, medium 12ep

Assume the price is per hour, and started counting when the firefighter arrive.

In [6]:
import pandas as pd
from datetime import datetime, timedelta
import math

# -------------------------------
# Resource Specifications and Constants
resources_catalog = {
    "Smoke Jumpers": {"deployment": 30*60, "cost": 5000, "units": 5, "ep": 5},
    "Fire Engines":  {"deployment": 60*60, "cost": 2000, "units": 10, "ep": 2},
    "Helicopters":   {"deployment": 45*60, "cost": 8000, "units": 3, "ep": 8},
    "Tanker Planes": {"deployment": 120*60, "cost": 15000, "units": 2, "ep": 15},
    "Ground Crews":  {"deployment": 90*60, "cost": 3000, "units": 8, "ep": 3},
}

# Define which teams are limited (max 50% of total available per fire)
limited_teams = {"Smoke Jumpers", "Fire Engines", "Ground Crews"}

# Calculate maximum deployable units per fire for limited teams.
limited_limits = {rtype: math.floor(props["units"] * 0.5) for rtype, props in resources_catalog.items() if rtype in limited_teams}

# Handling durations in hours per fire severity.
handling_duration = {"low": 4, "medium": 12, "high": 36}

# Required efficiency points per fire severity.
required_ep = {"low": 4, "medium": 12, "high": 36}

# Damage costs for missed fires.
damage_costs = {"low": 50000, "medium": 100000, "high": 200000}

# -------------------------------
# Build the resource pool: each unit is an entry with its next available time.
def build_resource_pool():
    pool = []
    for rtype, props in resources_catalog.items():
        for i in range(props["units"]):
            pool.append({
                "resource": rtype,
                "deployment": props["deployment"], # seconds
                "cost": props["cost"],             # per hour cost
                "ep": props["ep"],
                "next_available": datetime.min    # initially available
            })
    return pool

# -------------------------------
# Simulation function with detailed assignment logging.
def simulate_fire_response(fire_df, resource_pool):
    """
    For each fire event (ordered by fire_start_time, with high severity prioritized),
    select a combination of available resource units (prioritizing units with lowest 
    deployment times) until the cumulative ep meets the required threshold.
    
    For limited teams, allow at most 50% of the total units (pre-calculated in limited_limits) per event.
    For each assigned unit:
      - Arrival time = fire_start_time + deployment time.
      - Operating cost incurred = cost per hour * handling duration.
      - Update unit's next_available to (arrival time + handling duration).
    
    If resources cannot collectively meet the required ep, the fire is considered missed.
    
    Returns a report dictionary and detailed assignment log.
    """
    # Report counters.
    addressed = {"low": 0, "medium": 0, "high": 0}
    missed = {"low": 0, "medium": 0, "high": 0}
    total_operational_cost = 0
    total_damage_cost = 0
    assignment_log = []  # To log details for each fire event.

    # Ensure fire_start_time is datetime.
    fire_df['fire_start_time'] = pd.to_datetime(fire_df['fire_start_time'])
    
    # Sort events by fire_start_time and then by severity (high first)
    severity_rank = {"high": 3, "medium": 2, "low": 1}
    fire_df['severity_rank'] = fire_df['severity'].map(severity_rank)
    fire_df = fire_df.sort_values(by=['fire_start_time', 'severity_rank'], ascending=[True, False]).reset_index(drop=True)
    
    # Process each fire event.
    for idx, event in fire_df.iterrows():
        event_time = event['fire_start_time']
        sev = event['severity']
        req_ep = required_ep[sev]
        handling_hours = handling_duration[sev]  # in hours
        
        # Prepare a dictionary to collect assignment details for the event.
        event_assignment = {
            "fire_index": idx,
            "fire_start_time": event_time,
            "severity": sev,
            "assignments": {}  # key: resource type, value: list of (arrival_time, cost) details.
        }
        
        # Get available resource units (next_available <= event_time).
        available_units = [u for u in resource_pool if u["next_available"] <= event_time]
        # Sort available units by deployment time (lowest first).
        available_units.sort(key=lambda u: u["deployment"])
        
        # Group available_units by resource type.
        resource_groups = {}
        for unit in available_units:
            rtype = unit["resource"]
            resource_groups.setdefault(rtype, []).append(unit)
        
        cumulative_ep = 0
        chosen_units = []  # List to store chosen units.
        
        # Now iterate over resource types in increasing order of deployment time.
        # Sort resource types based on their deployment time.
        resource_types_sorted = sorted(resource_groups.keys(), key=lambda r: resources_catalog[r]["deployment"])
        
        # For each resource type, try deploying up to allowed limit if limited; otherwise deploy all available.
        for rtype in resource_types_sorted:
            # Allowed count for this fire event.
            if rtype in limited_teams:
                allowed = limited_limits[rtype]
            else:
                allowed = len(resource_groups[rtype])
            
            # Get available units for this resource type.
            units = resource_groups[rtype][:allowed]
            # Try to add each unit.
            for unit in units:
                chosen_units.append(unit)
                cumulative_ep += unit["ep"]
                # Record assignment details: arrival time and cost.
                arrival_time = event_time + timedelta(seconds=unit["deployment"])
                cost_incurred = unit["cost"] * handling_hours
                event_assignment["assignments"].setdefault(rtype, []).append({
                    "arrival_time": arrival_time,
                    "operating_cost": cost_incurred
                })
                if cumulative_ep >= req_ep:
                    break
            if cumulative_ep >= req_ep:
                break

        if cumulative_ep >= req_ep:
            addressed[sev] += 1
            # For each chosen unit, update next_available (busy until arrival time + handling duration).
            for unit in chosen_units:
                arrival_time = event_time + timedelta(seconds=unit["deployment"])
                unit["next_available"] = arrival_time + timedelta(hours=handling_hours)
                total_operational_cost += unit["cost"] * handling_hours
        else:
            missed[sev] += 1
            total_damage_cost += damage_costs[sev]
            # Log that no assignment could meet the threshold.
            event_assignment["assignments"] = "Missed Response"
        
        assignment_log.append(event_assignment)
    
    report = {
        "Number of fires addressed": sum(addressed.values()),
        "Number of fires missed": sum(missed.values()),
        "Total operational costs": total_operational_cost,
        "Estimated damage costs": total_damage_cost,
        "Fire severity (addressed)": addressed,
        "Fire severity (missed)": missed,
        "Assignment Log": assignment_log
    }
    return report

# -------------------------------
# Run simulation directly on cleaned current wildfire data.
current_path = "/Users/duchuyta/Downloads/SAP-CONUHACKS-IX/Data/Preprocessed_1/current_wildfiredata_cleaned.csv"
current_df = pd.read_csv(current_path)

# Rebuild resource pool (fresh start for simulation).
resource_pool = build_resource_pool()

print("Current Data Simulation Report:")
current_report = simulate_fire_response(current_df.copy(), resource_pool)
for key, value in current_report.items():
    # For Assignment Log, print summaries only.
    if key == "Assignment Log":
        print("Assignment Log:")
        for log in value[:]:
            print(log)
    else:
        print(f"{key}: {value}")


Current Data Simulation Report:
Number of fires addressed: 32
Number of fires missed: 0
Total operational costs: 11728000
Estimated damage costs: 0
Fire severity (addressed): {'low': 14, 'medium': 11, 'high': 7}
Fire severity (missed): {'low': 0, 'medium': 0, 'high': 0}
Assignment Log:
{'fire_index': 0, 'fire_start_time': Timestamp('2023-12-31 23:07:00'), 'severity': 'low', 'assignments': {'Smoke Jumpers': [{'arrival_time': Timestamp('2023-12-31 23:37:00'), 'operating_cost': 20000}]}}
{'fire_index': 1, 'fire_start_time': Timestamp('2024-01-03 01:53:00'), 'severity': 'low', 'assignments': {'Smoke Jumpers': [{'arrival_time': Timestamp('2024-01-03 02:23:00'), 'operating_cost': 20000}]}}
{'fire_index': 2, 'fire_start_time': Timestamp('2024-01-03 09:29:00'), 'severity': 'high', 'assignments': {'Smoke Jumpers': [{'arrival_time': Timestamp('2024-01-03 09:59:00'), 'operating_cost': 180000}, {'arrival_time': Timestamp('2024-01-03 09:59:00'), 'operating_cost': 180000}], 'Helicopters': [{'arrival

In [7]:
import json
from datetime import datetime
import pandas as pd

def default_serializer(obj):
    if isinstance(obj, (datetime, pd.Timestamp)):
        return obj.isoformat()
    return str(obj)

output_path = "/Users/duchuyta/Downloads/SAP-CONUHACKS-IX/Data/Output/model1_logical.json"

with open(output_path, "w") as f:
    json.dump(current_report, f, default=default_serializer, indent=4)